In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from data import kfolds, train_test_split, classification_dataset
from train import train_model
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
# Hyperparameters
val_perc = 0.2
k = 10
patience = 15
batch_size = 16
learning_rate = 1e-5
dropout_rate = 0.6

In [3]:
lidc_dataset, lidc_total_samples = classification_dataset(
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    return_size=True,
)
lidc_total_samples

754

In [4]:
def build_model():
    input_small = keras.Input(SMALL_PATCH_SHAPE, name="input_small")
    x_small = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="small_selu_conv3d_1",
    )(input_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_1")(x_small)
    x_small = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="small_selu_conv3d_2",
    )(x_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_2")(x_small)
    x_small = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="small_selu_conv3d_3",
    )(x_small)
    x_small = keras.layers.MaxPooling3D((1, 2, 2), name="small_maxpool_3")(x_small)
    x_small = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="small_selu_conv3d_4",
    )(x_small)
    x_small = keras.layers.Flatten(name="flatten_small")(x_small)

    input_big = keras.Input(BIG_PATCH_SHAPE, name="input_big")
    x_big = keras.layers.MaxPooling3D((2, 2, 2), name="big_maxpool_0")(input_big)
    x_big = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="big_selu_conv3d_1",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_1")(x_big)
    x_big = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="big_selu_conv3d_2",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_2")(x_big)
    x_big = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="big_selu_conv3d_3",
    )(x_big)
    x_big = keras.layers.MaxPooling3D((1, 2, 2), name="big_maxpool_3")(x_big)
    x_big = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="big_selu_conv3d_4",
    )(x_big)
    x_big = keras.layers.Flatten(name="flatten_big")(x_big)

    x = keras.layers.concatenate([x_small, x_big], name="concatenate")
    x = SeluDense(128, name="selu_dense")(x)
    x = keras.layers.AlphaDropout(dropout_rate, name="alpha_dropout")(x)
    x = keras.layers.Dense(1, activation="sigmoid", name="final_dense")(x)

    cnn_3d = keras.Model(inputs=[input_small, input_big], outputs=x, name="3dcnn")

    return cnn_3d

In [5]:
metrics = [
    keras.metrics.AUC(name="auc"),
    keras.metrics.BinaryAccuracy(name="accuracy"),
]

In [6]:
train_dataset, val_dataset = train_test_split(lidc_dataset, test_perc=val_perc)
val_dataset = val_dataset.batch(1)
train_dataset = (
    train_dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
cnn_3d = build_model()
cnn_3d.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss="binary_crossentropy",
    metrics=metrics,
)

log_dir = f"logs/baseline-lidc"
model_fname = f"models/baseline-lidc.h5"
cnn_3d = train_model(
        cnn_3d,
        train_dataset,
        val_dataset,
        patience,
        "val_accuracy",
        model_fname,
        log_dir,
        verbose_checkpoint=1,
    )


Epoch 00001: val_accuracy improved from -inf to 0.76000, saving model to models/baseline-lidc.h5

Epoch 00002: val_accuracy did not improve from 0.76000

Epoch 00003: val_accuracy did not improve from 0.76000

Epoch 00004: val_accuracy improved from 0.76000 to 0.78000, saving model to models/baseline-lidc.h5

Epoch 00005: val_accuracy did not improve from 0.78000

Epoch 00006: val_accuracy improved from 0.78000 to 0.78667, saving model to models/baseline-lidc.h5

Epoch 00007: val_accuracy did not improve from 0.78667

Epoch 00008: val_accuracy did not improve from 0.78667

Epoch 00009: val_accuracy improved from 0.78667 to 0.80667, saving model to models/baseline-lidc.h5

Epoch 00010: val_accuracy did not improve from 0.80667

Epoch 00011: val_accuracy did not improve from 0.80667

Epoch 00012: val_accuracy did not improve from 0.80667

Epoch 00013: val_accuracy improved from 0.80667 to 0.81333, saving model to models/baseline-lidc.h5

Epoch 00014: val_accuracy improved from 0.81333 t

In [11]:
spie_dataset, spie_total_samples = classification_dataset(
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    return_size=True,
)
spie_total_samples

73

In [12]:
from collections import Counter

print(Counter(label.numpy()[0] for _, label in spie_dataset))

Counter({0: 37, 1: 36})


In [13]:
cnn_3d.evaluate(spie_dataset.batch(1), return_dict=True)

73/73 [==============================] - 0s 4ms/step - loss: 2.6478 - auc: 0.6329 - accuracy: 0.5890


{'loss': 2.647796154022217,
 'auc': 0.632882833480835,
 'accuracy': 0.5890411138534546}

In [ ]:
mean_metrics = {
    metric.name: keras.metrics.Mean(f"{metric.name}_mean", dtype=tf.float32)
    for metric in metrics
}
fold_id = 0
for train_val_dataset, test_dataset in tqdm(
    kfolds(k, dataset, cardinality=total_samples), total=k
):
    test_dataset = test_dataset.batch(1)
    train_dataset, val_dataset = train_test_split(train_val_dataset, test_perc=val_perc)
    val_dataset = val_dataset.batch(1)
    train_dataset = (
        train_dataset.batch(batch_size)
        .cache()  # must be called before shuffle
        .shuffle(buffer_size=64, reshuffle_each_iteration=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    cnn_3d = build_model()
    cnn_3d.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss="binary_crossentropy",
        metrics=metrics,
    )

    log_dir = f"logs/baseline-lidc-{fold_id}"
    model_fname = f"models/baseline-lidc-{fold_id}.h5"
    cnn_3d = train_model(
        cnn_3d,
        train_dataset,
        val_dataset,
        patience,
        "val_accuracy",
        model_fname,
        log_dir,
    )

    test_metrics = cnn_3d.evaluate(test_dataset, return_dict=True, verbose=0)

    print(f" {fold_id=} ".center(40, "="))
    for metric_name, metric_value in test_metrics.items():
        if metric_name in mean_metrics:
            print(f"{metric_name}: {metric_value}")
            mean_metrics[metric_name].update_state(metric_value)

    fold_id += 1

print(" mean ".center(40, "="))
for metric_name, metric_value in mean_metrics.items():
    print(f"{metric_name}: {metric_value.result().numpy()}")

In [ ]:
from config import *

train_dataset, val_dataset = train_test_split(dataset, test_perc=val_perc)
val_dataset = val_dataset.batch(1)
train_dataset = (
    train_dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

cnn_3d = build_model()
cnn_3d.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss="binary_crossentropy",
    metrics=metrics,
)

log_dir = f"logs/baseline-lidc-{fold_id}"
model_fname = f"models/baseline-lidc-{fold_id}.h5"
cnn_3d = train_model(
    cnn_3d,
    train_dataset,
    val_dataset,
    patience,
    "val_loss",
    model_fname,
    log_dir,
    verbose_training=True
)

dataset, total_samples = classification_dataset(
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    return_size=True,
)
print(total_samples)
dataset = dataset.batch(1)
cnn_3d.evaluate(dataset, return_dict=True, verbose=False)

In [ ]:
from collections import Counter
from itertools import islice

i = 2

for fold_id, (train_val_dataset, test_dataset) in enumerate(
    islice(kfolds(k, dataset, cardinality=total_samples), i, i+1), i
):
    print(Counter(label.numpy()[0] for _, label in test_dataset))
    test_dataset = test_dataset.batch(1)
    cnn_3d = keras.models.load_model(f"models/baseline-lidc-{fold_id}.h5")
    print(cnn_3d.evaluate(test_dataset, return_dict=True, verbose=0))

In [ ]:
patches, label = next(iter(test_dataset.skip(5)))
print(f"label: {label[0][0].numpy()}")
prediction = cnn_3d(patches, training=False)
print(f"prediction: {prediction[0][0].numpy()}")
plot_volume_animation(patches[0][0, :])